In [12]:
import pandas as pd
import numpy as np
import os
import statsmodels.api as sm
import pickle

In [48]:
## Finding Mean HQ and ER returns

state = pd.read_csv("/home/harshit/geography/district.csv")
state = list(set(state["STATE"]))
state.sort()
stateer = pd.read_csv("/home/harshit/geography/ercompanies.csv")
stateer=stateer.replace("\+ACY-","&",regex=True)
hq_path = "/home/harshit/geography/hqportfolios/"

for h in state:
    date = pd.DataFrame(columns=['Month'])
    month = pd.DataFrame(columns=['Month'])
    if(os.path.exists(hq_path+h+".csv")==True):
        hqread = pd.read_csv(hq_path+h+".csv")
        date = pd.DataFrame(columns=['Month'])
        for j in hqread["Company"]:
            j=j.upper()
            df = pd.read_csv("/home/harshit/geography/mret/"+j+".csv")
            df.rename(columns={"Return":j+"HQ"},inplace=True)
            date = pd.merge(date,df,left_on="Month", right_on="Month",how="outer",sort=False)
        date["meanHQ"] = round(np.mean(date,axis=1),2)
        
    
    if(h in stateer):
        comp=stateer[h][pd.notna(stateer[h])]
        month = pd.DataFrame(date["Month"])
        for i in comp:
            i=i.upper()
            ret = pd.read_csv("/home/harshit/geography/mret/"+i[0:-2]+".csv")
            ret.rename(columns={"Return":i+"ER"},inplace=True)

            if "14" in i:
                if("February-2014" in ret.Month.values):
                    ret=ret[ret.loc[ret["Month"]=="February-2014"].index.values[0]:ret.loc[ret["Month"]=="December-2014"].index.values[0]+1]
                    month=pd.merge(month,ret,on="Month",how="outer")
                else:
                    ret=ret[:ret.loc[ret["Month"]=="December-2014"].index.values[0]+1]
                    month=pd.merge(month,ret,on="Month",how="outer")


            if "15" in i:
                if("January-2015" in ret.Month.values):
                    ret=ret[ret.loc[ret["Month"]=="January-2015"].index.values[0]:ret.loc[ret["Month"]=="December-2015"].index.values[0]+1]
                    month=pd.merge(month,ret,on="Month",how="outer")
                else:
                    ret=ret[:ret.loc[ret["Month"]=="December-2015"].index.values[0]+1]
                    month=pd.merge(month,ret,on="Month",how="outer")

            if "16" in i:
                if("January-2016" in ret.Month.values):
                    ret=ret[ret.loc[ret["Month"]=="January-2016"].index.values[0]:ret.loc[ret["Month"]=="December-2016"].index.values[0]+1]
                    month=pd.merge(month,ret,on="Month",how="outer")
                else:
                    ret=ret[:ret.loc[ret["Month"]=="December-2016"].index.values[0]+1]
                    month=pd.merge(month,ret,on="Month",how="outer")


            if "17" in i:
                if("January-2017" in ret.Month.values):
                    ret=ret[ret.loc[ret["Month"]=="January-2017"].index.values[0]:ret.loc[ret["Month"]=="December-2017"].index.values[0]+1]
                    month=pd.merge(month,ret,on="Month",how="outer")
                else:
                    ret=ret[:ret.loc[ret["Month"]=="December-2017"].index.values[0]+1]
                    month=pd.merge(month,ret,on="Month",how="outer")


            if "18" in i:
                if("January-2018" in ret.Month.values):
                    ret=ret[ret.loc[ret["Month"]=="January-2018"].index.values[0]:ret.loc[ret["Month"]=="December-2018"].index.values[0]+1]
                    month=pd.merge(month,ret,on="Month",how="outer")
                else:
                    ret=ret[:ret.loc[ret["Month"]=="December-2018"].index.values[0]+1]
                    month=pd.merge(month,ret,on="Month",how="outer")        

        
        month["meanER"] = round(np.mean(month,axis=1),2)
        
        
    date = pd.merge(date,month,left_on="Month",right_on="Month",how="outer",sort=False)  
    date = date[:-3]
    if(len(date!=0)):
        date.to_csv("/home/harshit/geography/meanret/"+h+".csv",index=False)

In [2]:
factors = pd.read_csv("/home/harshit/geography/fourfactors.csv")

In [48]:
lis = os.listdir("/home/harshit/geography/meanret/")
for i in lis:
    data = pd.read_csv("/home/harshit/geography/meanret/"+i)
    if ("meanER" in data and "meanHQ" in data):
        merged = pd.merge(factors,data[["meanHQ","meanER"]],left_index=True,right_index=True)
        merged=merged.dropna(axis=0,how='any')
        merged["meanHQ"]=merged["meanHQ"]-merged["Rf"]
        merged["meanER"]=merged["meanER"]-merged["Rf"]
        X= merged[["SMB","HML","WML","Rm-Rf","meanHQ"]] #Making X variable for entropy included model
        X = sm.add_constant(X)
        y = merged["meanER"]
        model2= sm.OLS(y,X,hasconst=True).fit()   #model for carhart 4 factor
        model2_name = "/home/harshit/geography/models/"+i+".sav"
        pickle.dump(model2,open(model2_name,"wb"))
        
        me = merged.drop(["Month"],axis=1)
        corr = me.corr()
        heat=corr.style.background_gradient(cmap='coolwarm').set_precision(2)
        print("\n"+i)
        display(heat)


GUJARAT.csv


,SMB,HML,WML,Rm,Rf,Rm-Rf,meanHQ,meanER
SMB,1,0.16,0.039,0.16,0.34,0.16,0.026,-0.07
HML,0.16,1,-0.41,0.58,0.0002,0.58,-0.51,-0.4
WML,0.039,-0.41,1,-0.23,-0.17,-0.23,0.15,0.19
Rm,0.16,0.58,-0.23,1,0.22,1,-0.77,-0.89
Rf,0.34,0.0002,-0.17,0.22,1,0.2,-0.083,-0.23
Rm-Rf,0.16,0.58,-0.23,1,0.2,1,-0.77,-0.89
meanHQ,0.026,-0.51,0.15,-0.77,-0.083,-0.77,1,0.64
meanER,-0.07,-0.4,0.19,-0.89,-0.23,-0.89,0.64,1



DELHI.csv


,SMB,HML,WML,Rm,Rf,Rm-Rf,meanHQ,meanER
SMB,1,0.16,0.039,0.16,0.34,0.16,-0.093,-0.034
HML,0.16,1,-0.41,0.58,0.0002,0.58,-0.57,-0.43
WML,0.039,-0.41,1,-0.23,-0.17,-0.23,0.23,0.26
Rm,0.16,0.58,-0.23,1,0.22,1,-0.9,-0.88
Rf,0.34,0.0002,-0.17,0.22,1,0.2,-0.22,-0.27
Rm-Rf,0.16,0.58,-0.23,1,0.2,1,-0.9,-0.88
meanHQ,-0.093,-0.57,0.23,-0.9,-0.22,-0.9,1,0.83
meanER,-0.034,-0.43,0.26,-0.88,-0.27,-0.88,0.83,1



KARNATAKA.csv


...

,SMB,HML,WML,Rm,Rf,Rm-Rf,meanHQ,meanER
SMB,1,0.27,0.079,0.24,0.41,0.23,-0.18,0.15
HML,0.27,1,-0.51,0.53,0.14,0.53,-0.25,-0.33
WML,0.079,-0.51,1,-0.26,-0.15,-0.26,0.22,0.24
Rm,0.24,0.53,-0.26,1,0.37,1,-0.5,-0.46
Rf,0.41,0.14,-0.15,0.37,1,0.36,-0.47,0.018
Rm-Rf,0.23,0.53,-0.26,1,0.36,1,-0.5,-0.46
meanHQ,-0.18,-0.25,0.22,-0.5,-0.47,-0.5,1,0.26
meanER,0.15,-0.33,0.24,-0.46,0.018,-0.46,0.26,1



MAHARASHTRA.csv


,SMB,HML,WML,Rm,Rf,Rm-Rf,meanHQ,meanER
SMB,1,0.16,0.039,0.16,0.34,0.16,-0.0073,-0.027
HML,0.16,1,-0.41,0.58,0.0002,0.58,-0.42,-0.39
WML,0.039,-0.41,1,-0.23,-0.17,-0.23,0.24,0.099
Rm,0.16,0.58,-0.23,1,0.22,1,-0.9,-0.75
Rf,0.34,0.0002,-0.17,0.22,1,0.2,-0.22,-0.18
Rm-Rf,0.16,0.58,-0.23,1,0.2,1,-0.9,-0.75
meanHQ,-0.0073,-0.42,0.24,-0.9,-0.22,-0.9,1,0.66
meanER,-0.027,-0.39,0.099,-0.75,-0.18,-0.75,0.66,1



TAMIL NADU.csv


,SMB,HML,WML,Rm,Rf,Rm-Rf,meanHQ,meanER
SMB,1,0.16,0.039,0.16,0.34,0.16,-0.23,-0.082
HML,0.16,1,-0.41,0.58,0.0002,0.58,-0.26,-0.26
WML,0.039,-0.41,1,-0.23,-0.17,-0.23,0.2,-0.016
Rm,0.16,0.58,-0.23,1,0.22,1,-0.46,-0.75
Rf,0.34,0.0002,-0.17,0.22,1,0.2,-0.074,-0.22
Rm-Rf,0.16,0.58,-0.23,1,0.2,1,-0.46,-0.74
meanHQ,-0.23,-0.26,0.2,-0.46,-0.074,-0.46,1,0.2
meanER,-0.082,-0.26,-0.016,-0.75,-0.22,-0.74,0.2,1


In [56]:
lis = os.listdir("/home/harshit/geography/meanret/")
for i in lis:
    data = pd.read_csv("/home/harshit/geography/meanret/"+i)
    if ("meanER" in data and "meanHQ" in data):
        merged = pd.merge(factors,data[["meanHQ","meanER"]],left_index=True,right_index=True)
        merged=merged.dropna(axis=0,how='any')
        merged["meanHQ"]=merged["meanHQ"]-merged["Rf"]
        merged["meanER"]=merged["meanER"]-merged["Rf"]
        X= merged[["SMB","HML","WML","Rm-Rf"]] #Making X variable for entropy included model
        X = sm.add_constant(X)
        y = merged["meanER"]
        model2= sm.OLS(y,X,hasconst=True).fit()   #model for carhart 4 factor
        model2_name = "/home/harshit/geography/nonhqmodels/"+i+".sav"
        pickle.dump(model2,open(model2_name,"wb"))

In [59]:
for i in os.listdir("/home/harshit/geography/nonhqmodels"):
    model1 = pickle.load(open("/home/harshit/geography/nonhqmodels/"+i,"rb"))
    model2 = pickle.load(open("/home/harshit/geography/models/"+i,"rb"))
    print(i.split(".")[0])
    print(model1.summary())
    print(model2.summary())

MAHARASHTRA
                            OLS Regression Results                            
Dep. Variable:                 meanER   R-squared:                       0.579
Model:                            OLS   Adj. R-squared:                  0.548
Method:                 Least Squares   F-statistic:                     18.56
Date:                Wed, 20 Mar 2019   Prob (F-statistic):           1.20e-09
Time:                        05:08:57   Log-Likelihood:                -156.44
No. Observations:                  59   AIC:                             322.9
Df Residuals:                      54   BIC:                             333.3
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.7852      0.509     -1.